In [ ]:
from requests import Session
from tabulate import tabulate
import MatifyAPI
import json
import io
# encoding=utf8

session = Session()
tokenResponse = session.post('https://api.etilbudsavis.dk/v2/sessions?api_key=00izwus560vpbdasf0o92m2fhaim4ynr')
token = (json.loads(tokenResponse.text))["token"]
if not token:
    print "Couldn't get token"
defaultHeaders = {'X-token': token, 'Origin': "https://etilbudsavis.dk/"}

In [ ]:
#List Store. 
listResponse = session.get('https://api.etilbudsavis.dk/v2/stores?r_lat=59.95&r_lng=10.75&r_radius=700000&api_av=0.1.33&limit=100',
                              headers=defaultHeaders)
stores = json.loads(listResponse.text)
storeList = [[store['branding']['name'], store['id']] for store in stores]
print "-----------------List 100 stores-----------------------------"
print tabulate(storeList, headers=['All Stores', 'Id'])
#Should use the branding name instead of store id to filter
#selectedId = ['7edbLOE', '90f2F7y', 'd8dewIy', '2665CXz', '82a9zXz', 'b88bQ7y', '618fq21', '98c86Iq']
#selectedStoreList = [[store['branding']['name'], store['id']] for store in stores if (store['id'] in selectedId)]
#print tabulate(selectedStoreList, headers=['Selected Stores', 'Id'])

#Note: input names in lower-case
#selectedBrandNames = ['coop mega', 'coop extra', 'coop marked', 'kiwi', 'meny', 'rema', 'matkroken', 
#                      'coop obs', 'coop prix','joker', 'bunnpris', 'extra', 'spar']
selectedBrandNames = ['spar']

In [ ]:
#List Catalog
#The store_ids filter of the API does not seem to work, so we have to filter ourself!
#storeIdStr = ','.join([selectedStore[1] for selectedStore in selectedStoreList])
catalogsResponse = session.get('https://api.etilbudsavis.dk/v2/catalogs?r_lat=59.95'+
                               '&r_lng=10.75&r_radius=700000&api_av=0.1.33&limit=100',
                               headers=defaultHeaders)
catalogs = json.loads(catalogsResponse.text)
catalogList = [[catalog['branding']['name'], catalog['store_id'], catalog['id']] for catalog in catalogs]
print tabulate(catalogList, headers=['All Catalogs', 'Store Id', 'Id'])

#Check if any the brand name contains any selectedBrandName
selectedCatalogs = [catalog for catalog in catalogs 
                    if len([brandName for brandName in selectedBrandNames if brandName in catalog['branding']['name'].lower()]) > 0]
selectedCatalogList = [[catalog['branding']['name'], catalog['id'], catalog['run_from'], 
                        next(brandName for brandName in selectedBrandNames if brandName in catalog['branding']['name'].lower())] 
                       for catalog in selectedCatalogs]
print "\n---------------------------------"
print tabulate(selectedCatalogList, headers=['Selected Catalogs', 'Id', 'Run From', 'Brand'])

In [ ]:
#Get offers in selected catalog
for catalogStore, catalogId, catalogRunFrom, catalogBrand in selectedCatalogList:
    print "--------------------------------------------"
    print "Requesting catalog "+catalogStore+" with id "+catalogId+"..."
    offersResponse = session.get('https://api.etilbudsavis.dk/v2/catalogs/'+catalogId+'/hotspots?r_lat=59.95&r_lng=10.75&r_radius=700000&api_av=0.1.33',
                        headers=defaultHeaders)
    offers = json.loads(offersResponse.text)
    print "Number of offers: "+ str(len(offers))
    if len(offers)==0:
        continue
    offerIdList = [offer['id'] for offer in offers]
    print "Parsing offers..."
    JsonFileName = catalogBrand + '_'+catalogId + "_" + catalogRunFrom + ".json"
    offerDescList = []
    for offerId in offerIdList:
        offerDescResponse = session.get('https://api.etilbudsavis.dk/v2/offers/'+offerId+'?r_lat=59.95&r_lng=10.75&r_radius=700000&api_av=0.1.33',
                                    headers=defaultHeaders)
        offerDescList.append(json.loads(offerDescResponse.text.encode('utf-8')))
    with io.open(JsonFileName, 'w', encoding="utf-8") as json_file:
        data = json.dumps(offerDescList, ensure_ascii=False)     
        json_file.write(unicode(data))
        MatifyAPI.uploadCatalogs(session = session, catalogId=catalogId, catalogJson=data, catalogName=JsonFileName)
    print "Done."

## Download deals from before

In [ ]:
selectedBrandNames = ['coop mega', 'coop extra', 'coop marked', 'kiwi', 'meny', 'rema', 'matkroken', 
                      'coop obs', 'coop prix','joker', 'bunnpris', 'extra', 'spar']

catalogsResponse = session.get('https://api.etilbudsavis.dk/v2/catalogs?r_lat=59.95'+
                               '&r_lng=10.75&r_radius=700000&api_av=0.1.33'+
                               '&offset=10&limit=100'+
                               '&run_from=2015-01-30T23:00:00+0000'+
                               '&run_till=2015-02-13T22:59:59+0000',
                               headers=defaultHeaders)
catalogs = json.loads(catalogsResponse.text)
catalogList = [[catalog['branding']['name'], catalog['store_id'], catalog['id']] for catalog in catalogs]
print tabulate(catalogList, headers=['All Catalogs', 'Store Id', 'Id'])

#Check if any brand name contains any selectedBrandName
selectedCatalogs = [catalog for catalog in catalogs 
                    if len([brandName for brandName in selectedBrandNames if brandName in catalog['branding']['name'].lower()]) > 0]
selectedCatalogList = [[catalog['branding']['name'], catalog['id'], catalog['run_from'], 
                        next(brandName for brandName in selectedBrandNames if brandName in catalog['branding']['name'].lower())] 
                       for catalog in selectedCatalogs]
print "\n---------------------------------"
print tabulate(selectedCatalogList, headers=['Selected Catalogs', 'Id', 'Run From', 'Brand'])

In [ ]:
catalogsResponse

In [ ]:
catalogs = json.loads(catalogsResponse.text)
